In [ ]:
import os
import tavily
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END
# To install: pip install tavily-python
from tavily import TavilyClient
from IPython.display import display_markdown


In [ ]:
load_dotenv()
client = TavilyClient(os.getenv("TAVILY_API_KEY"))

In [ ]:
from typing import TypedDict
class AgentState(TypedDict):
    question: str
    search_results: dict  
    answer: str

In [ ]:
def search_web(state: AgentState):
    """
    Node 1: Intelligent Web Searcha

    Uses Tavily's AI-optimized search to find and process web information.
    Behind the scenes: Tavily searches multiple sources, extracts relevant content,
    and uses AI to synthesize the information into a coherent answer.
    """
    #print(f"🔍 Searching: {state['question']}")
    client = TavilyClient(os.getenv("TAVILY_API_KEY"))
    
    
    search_results = client.search(
        query=state["question"],
        max_results=3,        
        include_answer=True      
    )

    return {"search_results": search_results}

In [ ]:
def generate_answer(state: AgentState):
    """
    Node 2: Answer Synthesis and Formatting

    Takes the search results from Tavily and formats them into a clean,
    user-friendly response with proper source attribution.
    """
    #print("🤖 Formatting answer...")

    # Extract Tavily's AI-generated answer (the smart synthesis)
    ai_answer = state["search_results"].get("answer", "No answer found")

    # Extract source URLs for transparency and verification
    sources = [f"- {result['title']}: {result['url']}" 
              for result in state["search_results"]["results"]]

    # Combine the intelligent answer with source attribution
    final_answer = f"{ai_answer}\n\nSources:\n" + "\n".join(sources)

    return {"answer": final_answer}

In [ ]:
def format_answer(state: AgentState):
    """
    Generate a clean, readable summarized version of the news from the agent state.
    This function:
      - Extracts search terms and summaries from the state.
      - Builds a formatted text summary for display or chat output.
    """

    # Extract data safely
    query = state.get("news_query", "General News")
    tldr_articles = state.get("tldr_articles") or []
    formatted_results = state.get("formatted_results", "")

    # If the workflow already produced formatted results, return that directly
    if formatted_results and formatted_results != "No articles with text found.":
        return formatted_results

    # If summaries exist, combine them into a readable final summary
    if tldr_articles:
        summary_lines = [f"📰 **{query.upper()} – Summary of Top {len(tldr_articles)} Articles**\n"]
        for i, article in enumerate(tldr_articles, 1):
            title = article.get("title", "Untitled Article")
            url = article.get("url", "")
            summary = article.get("summary", "No summary available.")
            summary_lines.append(f"**{i}. {title}**\n{url}\n{summary}\n")
        return "\n".join(summary_lines)

    # If no summaries exist, fall back to a default message
    return f"No summarized articles found for '{query}'. Please try a different search term."


In [ ]:
from langgraph.graph import StateGraph, END

def create_agent():
    """
    Build the AI Agent Workflow

    This creates a StateGraph where:
    - Each node is an independent function that can read/update shared state
    - LangGraph handles orchestration, state management, and execution flow
    - Easy to extend: just add more nodes and define their connections
    """
    # Create the workflow graph
    workflow = StateGraph(AgentState)

    # Define our processing nodes
    # "search" should be your node that gathers information (e.g., search_web)
    workflow.add_node("search", search_web)        # Step 1: Gather information  
    # "result" will format the final answer (uses the format_answer implementation)
    workflow.add_node("result", format_answer)     # Step 2: Process and format

    # Define the flow of intelligence
    workflow.set_entry_point("search")      # Start here
    workflow.add_edge("search", "result")   # After search, go to answer
    workflow.add_edge("result", END)        # After answer, we're done

    return workflow.compile()


In [ ]:
agent = create_agent()

In [ ]:
output = agent.invoke({"question":"todays weather"})
display_markdown(output["answer"], raw=True)